In [2]:
from typing import List
import torch
import torch.nn as nn


class ForLoop(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, xs: List[torch.Tensor]):
        while xs[0].numel() > 0:
            xs[0] = torch.zeros((0, 0))
        return xs


for_loop_model = ForLoop()
s_for_loop_model = torch.jit.script(for_loop_model)
print(s_for_loop_model.graph)


graph(%self : __torch__.___torch_mangle_5.ForLoop,
      %xs.1 : Tensor[]):
  %13 : NoneType = prim::Constant()
  %2 : int = prim::Constant[value=9223372036854775807]() # /tmp/ipykernel_1850367/2617197950.py:11:8
  %4 : int = prim::Constant[value=0]() # /tmp/ipykernel_1850367/2617197950.py:11:17
  %24 : Tensor = aten::__getitem__(%xs.1, %4) # /tmp/ipykernel_1850367/2617197950.py:11:14
  %25 : int = aten::numel(%24) # /tmp/ipykernel_1850367/2617197950.py:11:14
  %26 : bool = aten::gt(%25, %4) # /tmp/ipykernel_1850367/2617197950.py:11:14
   = prim::Loop(%2, %26) # /tmp/ipykernel_1850367/2617197950.py:11:8
    block0(%9 : int):
      %12 : int[] = prim::ListConstruct(%4, %4)
      %17 : Tensor = aten::zeros(%12, %13, %13, %13, %13) # /tmp/ipykernel_1850367/2617197950.py:12:20
      %19 : Tensor[] = aten::_set_item(%xs.1, %4, %17) # /tmp/ipykernel_1850367/2617197950.py:12:12
      %5 : Tensor = aten::__getitem__(%xs.1, %4) # /tmp/ipykernel_1850367/2617197950.py:11:14
      %6 : int = aten:

In [1]:
from typing import List
import torch
import torch.nn as nn
from brt.routers.app import RandScatterRouter
from brt.routers import GatherRouter
from brt.primitive.helper import symbolize


class ForLoop(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter = RandScatterRouter(dst_num=2)
        self.moe_decoder = nn.Linear(10, 10)
        self.gather = GatherRouter(dst_num=2)

    def forward(self, x: torch.Tensor):
        target = torch.zeros((1, 1), device=x.device)
        while target.numel() > 0:
            route_results = self.scatter(x)
            target = route_results[0]
            x = self.moe_decoder(route_results[0])
            x = self.gather([x, route_results[1]])
        return x


for_loop_model = ForLoop()

x = torch.randn(1, 10)
y = for_loop_model(x)
print(y)
s_for_loop_model = torch.jit.script(symbolize(for_loop_model))
print(s_for_loop_model.graph)


tensor([[ 0.0316, -0.0851, -0.8204, -0.9239, -0.1884,  0.0952, -0.3459, -0.1072,
          1.1982, -0.0687]], grad_fn=<AliasBackward0>)


RuntimeError: Could not cast attribute 'stream' to type Stream: Expected a value of type 'Stream' for field 'stream', but found 'int'
Exception raised from setattr at /opt/conda/conda-bld/pytorch_1646755903507/work/torch/csrc/jit/api/object.h:67 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x4d (0x7f238b6831bd in /state/partition/whcui/tools/pyenv/versions/miniconda3-3.8-4.10.3/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x68 (0x7f238b67f838 in /state/partition/whcui/tools/pyenv/versions/miniconda3-3.8-4.10.3/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x4e0c9d (0x7f240e4b5c9d in /state/partition/whcui/tools/pyenv/versions/miniconda3-3.8-4.10.3/lib/python3.8/site-packages/torch/lib/libtorch_python.so)
frame #3: <unknown function> + 0x6411f1 (0x7f240e6161f1 in /state/partition/whcui/tools/pyenv/versions/